### En este notebook voy a intentar clasificar las ubicaciones entre validas o no

In [1]:
import pandas as pd
import re
import numpy as np
import pickle
import nltk
import xgboost as xgb
import matplotlib.pyplot as plt
import lightgbm as lgb
from catboost import CatBoostRegressor
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import STOPWORDS
%matplotlib inline

#nltk.download('punkt')


In [2]:
train = pd.read_csv('csv/train.csv')
test  = pd.read_csv('csv/test.csv')

In [3]:
train['location'].isnull().mean() # tenemos un 30% de locations que son nul

0.33272034677525286

In [4]:
train['loc_nula'] = train.apply(lambda x: ( 1 if pd.isna(x['location']) else 0  ), axis = 1 )
test['loc_nula'] = test.apply(lambda x: ( 1 if pd.isna(x['location']) else 0  ), axis = 1 )

In [5]:
test.sample(5)

,id,keyword,location,text,loc_nula
3128,10370,weapons,NaN,NAVY SAYS LT CMDR WHO FIRED SIDEARM IN DEFENSE...,1
1288,4231,drowned,Jupiter,#dw_english Hundreds feared drowned as migrant...,0
653,2135,catastrophe,#BVSTRONG,'Now I am quietly waiting for the catastrophe ...,0
104,351,annihilation,NaN,Allied Plans for the Annihilation of the Germa...,1
1023,3354,demolition,TO YOUR CHOSEN DESTINATION,http://t.co/AAffnDCDNq Draw Day Demolition: Da...,0


In [6]:
### Esto ya lo habiamos hecho, ahora voy a agregar un nuevo 
### feature donde diga si la location es valida o no (si se encuentra en un txt con ubicaciones)

In [7]:
#traigo el csv de ubicaciones
ubicaciones = pd.read_csv('csv/worldcities.csv')

In [8]:
ubicaciones.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,19354922.0,1840034016
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519


In [9]:
# quiero una lista que contenga todas las ciudades con todos los nombres posibles.

In [10]:
ubicaciones_serie = pd.Series(ubicaciones['city_ascii']  )
ubicaciones_serie = ubicaciones_serie.append(ubicaciones['country'], ignore_index=True )
ubicaciones_serie  = ubicaciones_serie.append(ubicaciones['iso2'], ignore_index=True  )
ubicaciones_serie = ubicaciones_serie.append(ubicaciones['iso3'], ignore_index=True )

In [11]:
ubicaciones_serie

0              Tokyo
1           New York
2        Mexico City
3             Mumbai
4          Sao Paulo
            ...     
61967            GRL
61968            UKR
61969            RUS
61970            RUS
61971            CAN
Length: 61972, dtype: object

In [12]:
ubicaciones_serie[3]

'Mumbai'

In [13]:
ubicaciones_serie = ubicaciones_serie.drop_duplicates()

In [14]:
ubicaciones_serie

0              Tokyo
1           New York
2        Mexico City
3             Mumbai
4          Sao Paulo
            ...     
60228            TCA
60712            CYM
61404            COK
61626            FLK
61913            SGS
Length: 14128, dtype: object

In [15]:
ubicaciones_serie = ubicaciones_serie.apply(lambda x: str(x).replace(" ", "") )

In [16]:
ubicaciones_serie = ubicaciones_serie.apply(lambda x: str(x).lower() )

In [17]:
ubicaciones_serie

0             tokyo
1           newyork
2        mexicocity
3            mumbai
4          saopaulo
            ...    
60228           tca
60712           cym
61404           cok
61626           flk
61913           sgs
Length: 14128, dtype: object

In [18]:
### ahora tengo que separar las ciudades de mi csv de train.
ubicaciones_train = train['location'].copy()

In [19]:
ubicaciones_train = ubicaciones_train.fillna("invalida") #esto va a servir para filtrar.

In [20]:
ubicaciones_train = ubicaciones_train.apply(lambda x: x.lower() )

In [21]:
print(ubicaciones_train[6663], ";", ubicaciones_train[999] )

ohio, usa ; former yugoslav republic of macedonia


In [22]:
ubicaciones_train = ubicaciones_train.apply(lambda x: re.sub('[^A-Za-z0-9]+', '_', x) )

In [23]:
print(ubicaciones_train[6663], ";", ubicaciones_train[999] )

ohio_usa ; former_yugoslav_republic_of_macedonia


In [24]:
### Spliteo las ubicaciones del train, en espacios.
ubicaciones_train  = ubicaciones_train.apply(lambda x: x.split('_') )

In [25]:
print(ubicaciones_train[6663], ";", ubicaciones_train[999] )

['ohio', 'usa'] ; ['former', 'yugoslav', 'republic', 'of', 'macedonia']


In [80]:
ubicaciones_train.hasnans

False

In [27]:
print(ubicaciones_train[4771])

['reddit', '']


In [28]:
###cargo el w2v en memoria, voy a agregar un feature que sea la distancia cos en w2v
### a la ubicacion que mas se le parece.

In [29]:
# #descargar de aca https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz, pesa 1.5gb
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [30]:
def cos(x1, x2):
  return np.dot(x1, x2)/(np.linalg.norm(x1)*np.linalg.norm(x2))

In [54]:
def calcular_dist (ingresadas, validas, diccionario):
    
    maximo = 0.0
    distancia = 0.0
    
    vocabulario = diccionario.vocab
    
    for buscada in ingresadas :
        if (buscada != '' and buscada != 'invalida' and buscada in vocabulario) :
            for ubicacion in validas :
                if(ubicacion in vocabulario):
                    distancia = cos( diccionario[buscada], diccionario[ubicacion] )
                    if distancia > maximo : 
                        maximo = distancia

    return maximo

In [ ]:
new, york 

In [36]:
distancia_minima = [] #mientras mas cerca es a 1 la distancia mas parecido son los arrays

In [41]:
for ubicacion in ubicaciones_train :
    distancia = calcular_dist(ubicacion, ubicaciones_serie, word2vec)
    distancia_minima.append(distancia)


In [45]:
len(train)

7613

In [46]:
len(ubicaciones_train)

7613

In [59]:
print(- (len(ubicaciones_train) - len (distancia_minima) ) )

32


In [67]:
i = 0
for dist in distancia_minima :
    if dist !=0 :
        print (i,dist)
    i-= -1

31 0.99999994
63 0.99999994
64 1.0000001
65 0.67343485
66 1.0
67 1.0000001
69 0.69459975
71 0.5717257
72 0.39587674
74 0.47871852
78 1.0
80 0.39587674
81 0.71724564
82 1.0000001
83 1.0
85 1.0000001
86 1.0000001
87 0.99999994
88 0.789179
89 1.0
91 1.0000001
94 1.0
95 1.0000001
96 1.0000001
97 0.99999994
98 1.0000001
100 1.0
101 1.0000001
102 0.68217415
103 1.0000001
104 0.99999994
105 0.9999999
106 1.0
108 1.0
109 1.0000001
110 1.0
111 1.0
113 0.6517344
115 0.99999994
116 0.789179
118 1.0
120 1.0000001
121 1.0000001
123 1.0
125 0.5118446
126 1.0000001
127 1.0
128 1.0000001
129 1.0000001
130 0.68217415
132 0.68217415
133 1.0
136 1.0000001
138 1.0
143 1.0000001
145 1.0
146 1.0000001
147 1.0
149 0.36441994
150 1.0000001
151 1.0000001
152 0.9999999
153 1.0
156 0.49635184
159 1.0000001
161 1.0000001
169 0.58349824
170 0.8101705
171 1.0000001
173 1.0
174 1.0
175 1.0
176 1.0
177 0.58624816
178 0.49777803
180 0.9999999
181 0.6748817
182 0.99999994
183 0.6209566
185 1.0000001
186 0.7322954
187 0

2188 1.0000001
2189 1.0
2191 1.0000001
2192 0.38873172
2193 0.56605786
2194 0.40108067
2195 0.71724564
2198 0.99999994
2201 1.0
2203 0.33350298
2206 1.0
2208 0.618598
2209 1.0
2211 1.0
2212 1.0000001
2213 1.0000001
2217 1.0000001
2218 1.0
2219 0.6526317
2220 1.0
2221 1.0
2222 1.0000001
2224 1.0000001
2225 0.69459975
2226 0.6526317
2228 0.6526317
2230 0.99999994
2231 1.0000001
2232 0.5210406
2233 0.65658265
2234 0.7036151
2235 1.0
2237 1.0000001
2238 1.0000001
2241 0.9999999
2242 1.0000001
2243 1.0000001
2244 0.35442716
2245 0.99999994
2246 1.0000001
2248 1.0
2249 0.42597404
2253 1.0000001
2254 1.0000001
2256 0.99999994
2258 0.35442716
2263 1.0000001
2264 0.69459975
2266 0.6645197
2267 1.0000001
2268 0.5280522
2269 1.0000001
2271 1.0
2272 0.54292965
2274 1.0000001
2275 0.7738712
2277 1.0
2279 0.60602075
2282 1.0000001
2284 1.0
2285 1.0000001
2286 1.0
2289 1.0
2294 1.0
2297 1.0000001
2298 1.0
2300 0.5714079
2301 1.0
2302 1.0
2303 0.56605786
2304 0.71724564
2305 0.31965524
2310 0.52411175

4044 1.0000001
4050 1.0
4054 0.9999999
4056 1.0
4057 1.0
4058 1.0000001
4061 1.0
4062 1.0000001
4064 1.0
4067 0.59138155
4069 1.0
4070 1.0000001
4074 0.9999999
4075 1.0
4078 1.0000001
4083 1.0000001
4084 1.0000001
4086 0.99999994
4087 1.0000001
4088 0.9999999
4091 1.0
4093 0.6516661
4096 1.0000001
4097 1.0
4098 1.0
4099 0.58349824
4102 1.0000001
4103 0.6567876
4106 1.0000001
4107 0.49635184
4108 0.5956311
4109 0.6772914
4110 0.49635184
4112 1.0000001
4113 0.54292965
4115 0.99999994
4116 1.0
4117 1.0
4119 0.68928975
4121 1.0000001
4122 0.789179
4123 0.69127166
4124 0.43631095
4126 1.0000001
4127 1.0000001
4129 1.0000001
4131 0.49635184
4132 1.0000001
4133 1.0000001
4134 0.5118446
4135 0.52725655
4136 1.0
4137 0.49635184
4138 1.0000001
4139 0.99999994
4140 0.9999999
4141 1.0000001
4142 0.5329109
4144 0.45866606
4145 1.0000001
4147 1.0000001
4148 1.0000001
4149 1.0
4150 0.377864
4151 0.6285668
4152 0.5418005
4153 1.0000001
4156 1.0000001
4157 0.50961524
4159 1.0
4160 1.0000001
4161 0.6417

6170 1.0
6171 1.0
6172 0.32096705
6173 0.62307954
6174 0.5533102
6175 0.5717257
6182 0.99999994
6184 0.54292965
6187 1.0000001
6189 1.0000001
6190 0.39202404
6195 1.0000001
6196 1.0
6197 0.51400024
6200 0.9999999
6201 0.6645197
6203 0.99999994
6204 0.70744467
6205 0.59138155
6207 0.99999994
6208 1.0000001
6209 1.0
6210 0.70744467
6211 1.0
6212 1.0
6213 1.0
6214 1.0000001
6215 0.9999999
6217 1.0
6218 1.0000001
6219 1.0000001
6220 0.68928975
6221 1.0
6222 0.9999999
6225 1.0000001
6229 0.99999994
6230 1.0
6231 0.47206926
6235 1.0
6239 1.0
6244 1.0
6247 0.44403315
6248 1.0000001
6251 0.37991667
6252 1.0
6255 0.4650461
6256 0.64137655
6258 1.0
6259 0.9999999
6260 1.0
6261 1.0
6262 1.0
6264 0.37219697
6265 1.0000001
6266 0.99999994
6267 1.0000001
6268 1.0000001
6269 0.99999994
6270 1.0000001
6271 0.9999999
6272 0.99999994
6274 1.0000001
6275 1.0000001
6276 1.0
6277 1.0
6278 1.0000001
6279 0.99999994
6281 0.6458915
6282 0.789179
6283 0.66057885
6285 1.0
6286 1.0000001
6288 1.0
6289 1.0
6292 1

In [79]:
train.iloc[130]

id                                  187
keyword                      aftershock
location                            NaN
text        @OnFireAnders I love you bb
target                                0
loc_nula                              1
Name: 130, dtype: object